# Precompute all features

In [ ]:
%load_ext autoreload
%autoreload 2
import pickle
import nibabel as nib
import numpy as np 
import matplotlib.pyplot as plt
from skimage import measure # For marching cubes
import polyscope as ps # For mesh display
from persim import plot_diagrams, PersistenceImager
import pandas as pd
import os
import sys
import skimage
import skimage.io
sys.path.append("../src")
import glob
from geomstats import *
from topostats import *
from kernels import *
from utils3d import *

In [ ]:
def load_dictionary(metadata_path):
    df = pd.read_csv(metadata_path)
    data = {}
    for index, row in df.iterrows():
        patient_id = row["ID"]
        patient_id = "M-0".join(patient_id.split("M-")) #File paths have an extra 0 in ID
        data[patient_id] = row
    del data["UCSF-PDGM-0541"] # Skip Patient 541 because segmentation file is empty
    return data

def argsort(seq):
    return np.array(sorted(range(len(seq)), key=seq.__getitem__), dtype=int)



metadata_path = "../Data/UCSF-PDGM-metadata_v2.csv"
data = load_dictionary(metadata_path) 
for patient in list(data.keys()):
    if not "Glio" in data[patient]["Final pathologic diagnosis (WHO 2021)"]:
        del data[patient]


all_patients = {}

## Step 1: Enumerate UCSF dataset
all_data_path = "../Data/UCSF-PDGM-v3"
for patient in os.listdir(all_data_path):
    patient_folder_path = os.path.join(all_data_path, patient)
    patient = patient[:-6]

    tumor_seg_path = patient_folder_path + "/" + patient_folder_path[-20:-6] + "_tumor_segmentation.nii.gz"
    if os.path.exists(tumor_seg_path) and patient in data:
        all_patients[patient] = {"tumor_seg_path":tumor_seg_path}

## Step 2: Enumerate TCGA dataset
all_data_path = "../Data/Pre-operative_TCGA_GBM_NIfTI_and_Segmentations"
for patient in os.listdir(all_data_path):
    patient_folder_path = os.path.join(all_data_path, patient)
    tumor_seg_path = glob.glob(patient_folder_path + os.path.sep + "*GlistrBoost_ManuallyCorrected*")
    if len(tumor_seg_path) > 0:
        all_patients[patient] = {"tumor_seg_path":tumor_seg_path[0]}

## Step 3: Enumerate Penn dataset
files = glob.glob("../Data/PennDataset/images_segm/*") + glob.glob("../Data/PennDataset/automated_segm/*")
for path in files:
    patient = path.split("/")[-1].split("_11_segm.nii.gz")[0]
    all_patients[patient] = {"tumor_seg_path":path}

        
print("len(all_patients)", len(all_patients))
## Step 4: Load masks and extract levelsets
iso_names = {1:"Necrotic", 2:"Edema", 4:"Main Tumor"} # What the labels actually mean
iso_levels = [2, 4, 1] # Column order of the labels
tumor_types = ["Edema", "Main Tumor", "Necrotic"]

for i, patient in enumerate(all_patients.values()):
    if i%10 == 0:
        print(".", end="")
    tumor_seg_path = patient["tumor_seg_path"]
    tumor_seg_nifti = nib.load(tumor_seg_path)
    tumor_seg_mat = tumor_seg_nifti.get_fdata()
    
    for k, level in enumerate(iso_levels):
        binary = tumor_seg_mat==level
        level_name = iso_names[level]
        B = crop_binary_volume(binary)
        patient["B{}".format(level_name)] = B
        X = binary_volume_2coords(binary)
        patient["X{}".format(level_name)] = X

data = all_patients
to_delete = []
for p in data:
    if not "XEdema" in data[p].keys():
        to_delete.append(p)
print(to_delete)
for p in to_delete:
    del data[p]
print(len(data))


## Total Persistences: Alpha

(TODO Later: Grab and sort)

In [ ]:
persistence_cutoff = 1

out_dir = "../preprocessed/alphatotal"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
for p in data:
    fileout = "{}/{}.pkl".format(out_dir, p)
    if os.path.exists(fileout):
        continue
    total = []
    for i, name in enumerate(iso_levels):
        name = iso_names[name]
        X = data[p]["X{}".format(name)]
        PDs = get_alpha_filtration_3d(X)
        PDs = remove_infinite(PDs)
        totali = []
        for k in range(len(PDs)):
            Ik = PDs[k]
            if Ik.size > 0:
                PDs[k] = Ik[Ik[:, 1]-Ik[:, 0] > persistence_cutoff, :]
                totali.append(np.sum(PDs[k][:, 1]-PDs[k][:, 0]))
            else:
                totali.append(0)
        total.append(totali)
    total = np.array(total)
    pickle.dump({"x":total}, open(fileout, "wb"))

## Total Persistences: Cubical

In [ ]:
all_kernels = []
all_kernels.append([gauss3d(w=3), gauss3d(w=5), gauss3d(w=7), laplacian3d(w=3), laplacian3d(w=5), laplacian3d(w=7)])
randpath = "../preprocessed/cubicalrandtotal/kernels.pkl"
if not os.path.exists(randpath):
    all_kernels.append(get_random_3d_kernels(5, 10))
else:
    all_kernels.append(pickle.load(open(randpath, "rb"))["kernels"])

all_dirs = ["../preprocessed/cubicaltotal", "../preprocessed/cubicalrandtotal"]


for kernels, out_dir in zip(all_kernels, all_dirs):
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    pickle.dump({"kernels":kernels}, open("{}/kernels.pkl".format(out_dir), "wb"))

    for p in data:
        fileout = "{}/{}.pkl".format(out_dir, p)
        if os.path.exists(fileout):
            continue
        total = []
        for i, name in enumerate(iso_levels):
            name = iso_names[level]
            B = data[p]["B{}".format(name)]
            PDs = []
            for kernel in kernels:
                PDs += get_binary_kernel_cubical_filtration(B, kernel)
            PDs = remove_infinite(PDs)
            for I in PDs:
                if I.size > 0:
                    total.append(np.sum(I[:, 1]-I[:, 0]))
                else:
                    total.append(0)
        total = np.array(total)
        pickle.dump({"x":total}, open(fileout, "wb"))

## Fractal Dimension

In [ ]:
dmin = 6
dmax = 12

out_dir = "../preprocessed/fractaldim"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
for p in data:
    fileout = "{}/{}.pkl".format(out_dir, p)
    if os.path.exists(fileout):
        continue
    dims = np.zeros(len(tumor_types))
    for i, tumor_type in enumerate(tumor_types):
        B = data[p]["B{}".format(tumor_type)]
        if np.sum(B) > 1: # There is at least one edge
            dims[i] = get_correlation_dimension_grid(B, dmin, dmax)
    pickle.dump({"x":dims}, open(fileout, "wb"))

## Shape Histograms

In [ ]:
r_max = 75
n_shells = 20

out_dir = "../preprocessed/shapehist"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
for p in data:
    fileout = "{}/{}.pkl".format(out_dir, p)
    if os.path.exists(fileout):
        continue
    hists = np.array([])
    for tumor_type in tumor_types:
        X = data[p]["X{}".format(tumor_type)]
        h = get_shape_hist(X, n_shells=n_shells, r_max=r_max)
        if hists.size == 0:
            hists = h
        else:
            hists = np.concatenate((hists, h))
    pickle.dump({"x":hists}, open(fileout, "wb"))

## Shape Shell Histograms

In [ ]:
"""
r_max = 75
n_shells = 20
subdiv = 1 # How many times to subdivide the sphere for sector points

out_dir = "../preprocessed/shapeshellhist"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
for p in data:
    fileout = "{}/{}.pkl".format(out_dir, p)
    if os.path.exists(fileout):
        continue
    hists = np.array([])
    for tumor_type in tumor_types:
        X = data[p]["X{}".format(tumor_type)]
        h = get_shape_shell_hist(X, n_shells=n_shells, r_max=r_max, subdiv=subdiv)
        print(h.shape)
        if hists.size == 0:
            hists = h
        else:
            hists = np.concatenate((hists, h))
    pickle.dump({"x":hists}, open(fileout, "wb"))
"""

## Shape PCA Histograms

In [ ]:
r_max = 75
n_shells = 10

out_dir = "../preprocessed/shapehistpca"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
for p in data:
    fileout = "{}/{}.pkl".format(out_dir, p)
    if os.path.exists(fileout):
        continue
    hists = np.array([])
    for tumor_type in tumor_types:
        X = data[p]["X{}".format(tumor_type)]
        h = get_shape_pca_hist(X, n_shells=n_shells, r_max=r_max)
        if hists.size == 0:
            hists = h
        else:
            hists = np.concatenate((hists, h))
    pickle.dump({"x":hists}, open(fileout, "wb"))

## D2 Histogram

In [ ]:
r_max = 75
n_bins = 40 # Number of bins in the histogram between [0, d_max]
n_samples = 10000 # Number of random samples

out_dir = "../preprocessed/d2"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
for p in data:
    fileout = "{}/{}.pkl".format(out_dir, p)
    if os.path.exists(fileout):
        continue
    hists = np.array([])
    for tumor_type in tumor_types:
        X = data[p]["X{}".format(tumor_type)]
        h = get_d2_hist(X, d_max=r_max*2, n_bins=n_bins, n_samples=n_samples)
        if hists.size == 0:
            hists = h
        else:
            hists = np.concatenate((hists, h))
    pickle.dump({"x":hists}, open(fileout, "wb"))

## Spin Images

In [ ]:
"""
r_max = 75
n_angles = 50
dim = 64

out_dir = "../preprocessed/spinimages"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
for p in data:
    fileout = "{}/{}.pkl".format(out_dir, p)
    if os.path.exists(fileout):
        continue
    imgs = np.array([])
    for tumor_type in tumor_types:
        X = data[p]["X{}".format(tumor_type)]
        img = get_spin_image(X, n_angles, r_max, dim).flatten()
        if imgs.size == 0:
            imgs = img
        else:
            imgs = np.concatenate((imgs, img))
    pickle.dump({"x":imgs}, open(fileout, "wb"))
"""

## Connected Components

In [ ]:
max_components = 10

out_dir = "../preprocessed/connectedcomponents"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
for p in data:
    fileout = "{}/{}.pkl".format(out_dir, p)
    if os.path.exists(fileout):
        continue
    all_counts = np.zeros(max_components*len(iso_names))
    for i, tumor_type in enumerate(tumor_types):
        B = data[p]["B{}".format(tumor_type)]
        B = crop_binary_volume(B)
        if B.size > 0:
            labels = label_volume_components(B, cluster_cutoff=1)
            counts = sorted(get_label_counts(labels).flatten())[::-1]
            i1 = i*max_components
            i2 = min((i+1)*max_components, i1+len(counts))
            all_counts[i1:i2] = counts[0:(i2-i1)]
    pickle.dump({"x":all_counts}, open(fileout, "wb"))

## Region Voxel Counts

(Can usually be inferred from connected component counts)

In [ ]:
out_dir = "../preprocessed/voxelcount"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
for p in data:
    fileout = "{}/{}.pkl".format(out_dir, p)
    if os.path.exists(fileout):
        continue
    counts = np.zeros(len(tumor_types))
    for i, tumor_type in enumerate(tumor_types):
        B = data[p]["B{}".format(tumor_type)]
        counts[i] = np.sum(B)
    pickle.dump({"x":counts}, open(fileout, "wb"))

# Condense Features

In [ ]:
from datasets import condense_dataset
features = ["alphatotal", "connectedcomponents", "cubicalrandtotal", "cubicaltotal", 
            "d2", "shapehist", "shapehistpca", "voxelcount", "fractaldim"]
data = {}
for feature in features:
    X, IDs = condense_dataset("../preprocessed/{}".format(feature), metadata_path)
    pickle.dump({"X":X, "IDs":IDs}, open("../preprocessed/{}.pkl".format(feature), "wb"))